In [1]:
import random
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')
import sys
sys.path.append('/content/gdrive/My Drive/bitnet')


Mounted at /content/gdrive/


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if __name__=='__main__':
    print('Using device:', device)

Using device: cpu


In [5]:
# !cat '/content/gdrive/My Drive/bitnet/processed/patients_mimic3_full.json'

In [6]:
import os
import data_proc
from data_proc import Dataset

In [7]:
dataset = Dataset()
data = dataset.load_data()
data = data
len(data)

Number of second level category:  170
Length of reverse dictionary  3875


7496

In [8]:
dataset.max_len_visit, dataset.vocabulary_size, dataset.digit3_size

(39, 3875, 170)

In [9]:
print(data[0][1])
print(data[0][4])

[1, 4, 31, 56, 61, 90, 105, 109, 114, 146]
1


In [10]:
pids = [i[0] for i in data]
intervals = [i[1] for i in data]
seqs = [i[2] for i in data]


readmission = [i[4] for i in data]
diag = [i[3] for i in data]

num_codes = set([code for visits in seqs for visit in visits for code in visit])
num_codes = len(set(num_codes)) 

print("num_codes",num_codes)


assert len(pids) == len(seqs) == len(intervals) == len(readmission)

num_codes 3434


In [11]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, seqs, intervals, readmission, diag):
        self.seqs = seqs
        self.intervals = intervals
        self.y1 = readmission
        self.y2 = diag
    
    def __len__(self):
        
        return len(self.y1)
    
    def __getitem__(self, index):

        return self.seqs[index], self.intervals[index], self.y1[index], self.y2[index]
data = CustomDataset(seqs, intervals, readmission, diag)
print(len(data))

7496


In [12]:
from torch.utils.data.dataset import random_split

train_test_split = int(len(data)*0.9)
lengths = [train_test_split, len(data) - train_test_split]
train_data, test_data = random_split(data, lengths)


train_val_split = int(len(train_data)*0.89)
lengths = [train_val_split, len(train_data) - train_val_split]
train_data, val_data = random_split(train_data, lengths)


print("Length of train dataset:", len(train_data))
print("Length of val dataset:", len(val_data))
print("Length of test dataset:", len(test_data))


Length of train dataset: 6003
Length of val dataset: 743
Length of test dataset: 750


In [13]:
def collate_fn(data):
  sequences, intervals, labels1, labels2 = zip(*data)

  num_patients = len(sequences)
  num_visits = len(sequences[0])
  num_codes = len(sequences[0][0])



  seq = torch.zeros((num_patients, num_visits, num_codes), dtype=torch.long)
  masks = torch.zeros((num_patients, num_visits, num_codes), dtype=torch.bool)
  y1 = torch.tensor(labels1, dtype=torch.float)
  y2 = torch.tensor(labels2, dtype=torch.float)
  itv = torch.zeros((num_patients, num_visits), dtype=torch.long)
  
  v_masks = torch.zeros((num_patients, num_visits), dtype=torch.bool)
  

  for i_patient, patient in enumerate(sequences):
        for j_visit, visit in enumerate(patient):            
            seq[i_patient][j_visit] = torch.tensor(sequences[i_patient][j_visit],dtype=torch.long)
            for k in range(num_codes):
              if seq[i_patient][j_visit][k] != 0:
                masks[i_patient][j_visit][k] = True
  for i_patient, patient in enumerate(intervals):
        for j_visit, visit in enumerate(patient):            
            itv[i_patient][j_visit] = torch.tensor(intervals[i_patient][j_visit],dtype=torch.long)
            if itv[i_patient][j_visit] != 0:
                v_masks[i_patient][j_visit]  = True
  return sequences, intervals, masks, v_masks, y1, y2

In [14]:
from torch.utils.data import DataLoader

def load_data(train_data, val_data, test_data, collate_fn):
    
    batch_size = 32
    
    train_loader = DataLoader(dataset = train_data, batch_size = 32, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(dataset = val_data, batch_size = 32, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(dataset = test_data, batch_size = 32, shuffle=False, collate_fn=collate_fn)

    
    return train_loader, val_loader, test_loader


train_loader, val_loader, test_loader = load_data(train_data, val_data, test_data, collate_fn)

print(num_codes)

3434


In [15]:
class SelfAttentionPooling(nn.Module):
    def __init__(self, input_dim):
        super(SelfAttentionPooling, self).__init__()
        self.W = nn.Linear(input_dim, 1)
    
    def forward(self, batch_rep):
        """
        input:
            batch_rep : size (N, T, H), N: batch size, T: sequence length, H: Hidden dimension
      
        attention_weight:
            att_w : size (N, T, 1)
    
        return:
            utter_rep: size (N, H)
        """
        softmax = nn.functional.softmax
        att_w = softmax(self.W(batch_rep).squeeze(-1)).unsqueeze(-1)
        utter_rep = torch.sum(batch_rep * att_w, dim=1)

        return utter_rep


In [16]:
class MultiHeadAttention_O(nn.Module):
    def __init__(self, direction, dropout, num_units, num_heads=10,  **kwargs):
        super(MultiHeadAttention, self).__init__(**kwargs)
        self.num_heads = num_heads
        self.direction = direction
        self.dropout = nn.Dropout(p=dropout)
        self.num_units = num_units
        self.q_linear = nn.LazyLinear(self.num_units, bias=False)
        self.k_linear = nn.LazyLinear(self.num_units, bias=False)
        self.v_linear = nn.LazyLinear(self.num_units, bias=False)

    def forward(self, inputs):

        # because of self-attention, queries and keys is equal to inputs
        input_tensor, input_mask = inputs
        # print("input_tensor.shape", input_tensor.shape)
        queries = input_tensor
        keys = input_tensor

        # Linear projections
        Q = self.q_linear(queries)  # (N, L_q, d)
        K = self.k_linear(keys)  # (N, L_k, d)
        V = self.v_linear(keys)  # (N, L_k, d)

        print('Q shape: ', Q.shape)

        # Q shape:  torch.Size([320, 39, 100])
        # outputs shape:  torch.Size([10, 39, 3200])
        # input_mask:  torch.Size([320, 39])
        # val_mask:  torch.Size([320, 39, 1])

        # Split and concat
        Q_ = torch.concat(torch.split(Q, self.num_heads, dim=2), axis=0)  # (h*N, L_q, d/h)
        K_ = torch.concat(torch.split(K, self.num_heads, dim=2), axis=0)  # (h*N, L_k, d/h)
        V_ = torch.concat(torch.split(V, self.num_heads, dim=2), axis=0)  # (h*N, L_k, d/h)

        # Multiplication
        outputs = torch.matmul(Q_, torch.transpose(K_, 2, 1))  # (h*N, L_q, L_k)

        # print('outputs m shape: ', outputs.shape)

        # Q shape:  torch.Size([320, 

        # Scale
        outputs = outputs / (list(K_.shape)[-1] ** 0.5) # (h*N, L_q, L_k)

        # print('outputs s shape: ', outputs.shape)/

        # Key Masking
        key_masks = torch.sign(torch.sum(torch.abs(K_), axis=-1))  # (h*N, T_k)
        key_masks = torch.unsqueeze(key_masks, 1)  # (h*N, 1, T_k)
        key_masks = torch.tile(key_masks, [1, list(Q_.shape)[1], 1])  # (h*N, T_q, T_k)

        # Apply masks to outputs
        paddings = torch.ones_like(outputs) * (-2 ** 32 + 1)  # exp mask
        outputs = torch.where(key_masks == 0, paddings, outputs) # (h*N, T_q, T_k)
        
        # print('outputs masked shape: ', outputs.shape)


        n_visits = input_tensor.shape[1]
        sw_indices = torch.arange(0, n_visits, dtype=torch.int32)
        sw_col, sw_row = torch.meshgrid(sw_indices, sw_indices)
        # if self.direction == 'diag':
        #     # shape of (n_visits, n_visits)
        #     attention_mask = torch.t(tf.linalg.diag(- tf.ones([n_visits], tf.int32)) + 1, tf.bool)
        # elif self.direction == 'forward':
        #     attention_mask = tf.greater(sw_row, sw_col)  # shape of (n_visits, n_visits)
        # else:
        attention_mask = torch.greater(sw_col, sw_row)  # shape of (n_visits, n_visits)
        adder = (1.0 - attention_mask.type(outputs.dtype)) * -10000.0

        outputs = outputs + adder

        # softmax
        softmax = nn.Softmax(dim=-1)  # (h*N, T_q, T_k)
        outputs = softmax(outputs)
        # print('outputs softmax shape: ', outputs.shape)


        # Query Masking
        query_masks = torch.sign(torch.sum(torch.abs(Q_), axis=-1))  # (h*N, T_q)
        query_masks = torch.unsqueeze(query_masks, -1)  # (h*N, T_q, 1)
        query_masks = torch.tile(query_masks, [1, 1, K_.shape[1]])  # (h*N, T_q, T_k)

        # print('query_masks shape: ', query_masks.shape)

        # Apply masks to outputs
        outputs = outputs * query_masks

        # print('outputs masked shape: ', outputs.shape)

        # Dropouts
        outputs = self.dropout(outputs)
        # Weighted sum
        print('matmul: ', outputs.shape, V_.shape)

        outputs = torch.matmul(outputs, V_)  # ( h*N, T_q, C/h)
        # print('outputs weighted sum shape: ', outputs.shape)


        outputs = torch.split(outputs, self.num_heads, dim=0)
        # print('outputs split shape: ', len(outputs), outputs[0].shape)

        # Restore shape
        outputs = torch.concat(outputs, axis=0)  # (N, L_q, d)

        # print('outputs restore shape: ', outputs.shape)

        # input padding
        # print('outputs shape: ', outputs.shape)
        # print('input_mask: ', input_mask.shape)
        val_mask = torch.unsqueeze(input_mask, -1)
        # print('val_mask: ', val_mask.shape)
        outputs = torch.mul(outputs, val_mask.type(torch.float32))

        return outputs

In [17]:
import math
class ScaleDotProductAttention(nn.Module):
    """
    compute scale dot product attention
    Query : given sentence that we focused on (decoder)
    Key : every sentence to check relationship with Qeury(encoder)
    Value : every sentence same with Key (encoder)
    """

    def __init__(self):
        super(ScaleDotProductAttention, self).__init__()
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None, e=1e-12):
        # input is 4 dimension tensor
        # [batch_size, head, length, d_tensor]
        
        batch_size, head, length, d_tensor = k.size()

        # 1. dot product Query with Key^T to compute similarity
        k_t = k.transpose(2, 3)  # transpose
        score = (q @ k_t) / math.sqrt(d_tensor)  # scaled dot product

        # 2. apply masking (opt)
        if mask is not None:
            score = score.masked_fill(mask == 0, -e)

        # 3. pass them softmax to make [0, 1] range
        score = self.softmax(score)

        # 4. multiply with Value
        v = score @ v

        return v, score

class MultiHeadAttention(nn.Module):

    def __init__(self, d_model, n_head):
        super(MultiHeadAttention, self).__init__()
        self.n_head = n_head
        self.attention = ScaleDotProductAttention()
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_concat = nn.Linear(d_model, d_model)

    def forward(self, inputs, mask=None):
        # 1. dot product with weight matrices
        input_tensor, input_mask = inputs
        # print("input_tensor.shape", input_tensor.shape)
        q = input_tensor
        k = input_tensor
        v = input_tensor
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        # 2. split tensor by number of heads
        q, k, v = self.split(q), self.split(k), self.split(v)

        # 3. do scale dot product to compute similarity
        out, attention = self.attention(q, k, v, mask=mask)

        # 4. concat and pass to linear layer
        out = self.concat(out)
        out = self.w_concat(out)

        # 5. visualize attention map
        # TODO : we should implement visualization

        return out

    def split(self, tensor):
        """
        split tensor by number of head
        :param tensor: [batch_size, length, d_model]
        :return: [batch_size, head, length, d_tensor]
        """
        batch_size, length, d_model = tensor.size()


        d_tensor = d_model // self.n_head
        tensor = tensor.view(batch_size, length, self.n_head, d_tensor).transpose(1, 2)
        # it is similar with group convolution (split by number of heads)

        return tensor

    def concat(self, tensor):
        """
        inverse function of self.split(tensor : torch.Tensor)
        :param tensor: [batch_size, head, length, d_tensor]
        :return: [batch_size, length, d_model]
        """
        batch_size, head, length, d_tensor = tensor.size()
        d_model = head * d_tensor

        tensor = tensor.transpose(1, 2).contiguous().view(batch_size, length, d_model)
        return tensor

In [18]:
class FeedForwardNetwork(nn.Module):
  """Fully connected feedforward network."""

  def __init__(self, hidden_size, filter_size, relu_dropout, allow_pad, **kwargs):
    super(FeedForwardNetwork, self).__init__(**kwargs)
    self.hidden_size = hidden_size
    self.filter_size = filter_size
    self.relu_dropout = relu_dropout
    self.allow_pad = allow_pad

    self.filter_dense_layer = nn.LazyLinear(filter_size)
    self.activation = nn.ReLU()
    self.output_dense_layer = nn.LazyLinear(hidden_size)
    self.dropout = nn.Dropout(p=self.relu_dropout)

  def forward(self, x):
    """Return outputs of the feedforward network.

    Args:
      x: tensor with shape [batch_size, length, hidden_size]

    Returns:
      Output of the feedforward network.
      tensor with shape [batch_size, length, hidden_size]
    """
    x, mask = x
    output = self.filter_dense_layer(x)
    output = self.dropout(output)
    output = self.output_dense_layer(output)

    # input padding
    # val_mask = tf.expand_dims(mask, -1)
    # output = tf.multiply(output, tf.cast(val_mask, tf.float32))

    return output

In [19]:
class EncoderStack(nn.Module):
    """Transformer encoder stack.

    The encoder stack is made up of N identical layers. Each layer is composed
    of the sublayers:
    1. Masked encoder layer
    2. Feedforward network (which is 2 fully-connected layers)
    """

    def __init__(self, params, embedding_size, **kwargs):
        super(EncoderStack, self).__init__(**kwargs)
        self.layers = []
        self.embedding_size = embedding_size
        self.num_hidden_layers = params["num_hidden_layers"]
        for _ in range(params["num_hidden_layers"]):
            # Create sublayers for each layer.
            # masked_encoder_layer = nn.MultiheadAttention(self.embedding_size, params["num_heads"])
            masked_encoder_layer = MultiHeadAttention(self.embedding_size,
                                                      params["num_heads"])
            feed_forward_network = FeedForwardNetwork(params["hidden_size"],
                                                      params["filter_size"],
                                                      params["dropout"],
                                                      params["allow_ffn_pad"])
            self.layers.append([masked_encoder_layer, feed_forward_network])

    def forward(self, inputs, input_mask):
        encoder_inputs, input_mask = inputs, input_mask
        """Return the output of the encoder layer stacks.

        Args:
          encoder_inputs: tensor with shape [batch_size, number_visits, number_codes, hidden_size]
          input_mask: mask for the encoder self-attention layer.
            [batch_size, number_visits, number_codes]

        Returns:
          Output of encoder layer stack.
          float32 tensor with shape [batch_size, inumber_visits, number_codes, hidden_size]
        """
        for n, layer in enumerate(self.layers):
          # Run inputs through the sublayers.
          masked_encoder_layer = layer[0]
          feed_forward_network = layer[1]

          encoder_inputs = masked_encoder_layer((encoder_inputs, input_mask))
          encoder_inputs = feed_forward_network((encoder_inputs, input_mask))

        return encoder_inputs

In [20]:
from functools import reduce
from operator import mul

class Flatten(nn.Module):
    def __init__(self, keep, **kwargs):
        super(Flatten, self).__init__(**kwargs)
        self.keep = keep

    def forward(self, inputs):
        fixed_shape = list(inputs.shape)
        start = len(fixed_shape) - self.keep
        left = reduce(mul, [fixed_shape[i] or input.shape[i] for i in range(start)])
        out_shape = [left] + [fixed_shape[i] or input.shape[i] for i in range(start, len(fixed_shape))]
        flat = torch.reshape(inputs, out_shape)
        return flat

In [21]:
class PositionalEncoding1D(nn.Module):
    def __init__(self, channels):
        """
        :param channels: The last dimension of the tensor you want to apply pos emb to.
        """
        super(PositionalEncoding1D, self).__init__()
        self.org_channels = channels
        channels = int(np.ceil(channels / 2) * 2)
        self.channels = channels
        inv_freq = 1.0 / (10000 ** (torch.arange(0, channels, 2).float() / channels))
        self.register_buffer("inv_freq", inv_freq)
        self.cached_penc = None

    def forward(self, tensor):
        """
        :param tensor: A 3d tensor of size (batch_size, x, ch)
        :return: Positional Encoding Matrix of size (batch_size, x, ch)
        """
        if len(tensor.shape) != 3:
            raise RuntimeError("The input tensor has to be 3d!")

        if self.cached_penc is not None and self.cached_penc.shape == tensor.shape:
            return self.cached_penc

        self.cached_penc = None
        batch_size, x, orig_ch = tensor.shape
        pos_x = torch.arange(x, device=tensor.device).type(self.inv_freq.type())
        sin_inp_x = torch.einsum("i,j->ij", pos_x, self.inv_freq)
        emb_x = torch.cat((sin_inp_x.sin(), sin_inp_x.cos()), dim=-1)
        emb = torch.zeros((x, self.channels), device=tensor.device).type(tensor.type())
        emb[:, : self.channels] = emb_x

        self.cached_penc = emb[None, :, :orig_ch].repeat(batch_size, 1, 1)
        return self.cached_penc
        
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, seq_len) -> None:
        super(PositionalEncoding, self).__init__()
        self.d_model = d_model

        pe = torch.zeros(seq_len, d_model)

        for pos in range(seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i) / d_model)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2 * (i+1)) / d_model)))

        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x) -> torch.Tensor:
        seq_len = x.shape[1]
        x = math.sqrt(self.d_model) * x
        x = x + self.pe[:, :seq_len].requires_grad_(False)
        return x

In [22]:
class BiteNet(nn.Module):

    def __init__(self, dataset):
        super(BiteNet, self).__init__()
        self.device = device
        self.lr = 0.001
        self.dropout_rate = 0.2
        self.n_intervals = 12 * 365 + 1
        self.n_visits = 10
        self.n_codes = dataset.max_len_visit
        self.vocabulary_size = dataset.vocabulary_size
        self.digit3_size = dataset.digit3_size
        self.embedding_size = 100
        self.num_hidden_layers = 5
        self.predict_type = "re"
        self.batch_size = 32
        self.num_heads = 10
        
        self.params = dict()
        self.params["hidden_size"] = self.embedding_size
        self.params["filter_size"] = self.embedding_size
        self.params["dropout"] = self.dropout_rate
        self.params["allow_ffn_pad"] = False
        self.params["num_hidden_layers"] = self.num_hidden_layers
        self.params["is_scale"] = False
        self.params["direction"] = 'diag'
        self.params["num_heads"] = self.num_heads

        
        self.flatten_1 = Flatten(1)
        self.flatten_2 = Flatten(2)
        
        self.embedding = nn.Embedding(self.vocabulary_size, self.embedding_size)
        self.interval_embedding = nn.Embedding(self.n_intervals, self.embedding_size)
        self.encoder_stack = EncoderStack(self.params, self.embedding_size)
        self.attention_pooling = SelfAttentionPooling(self.embedding_size)
        self.fc1 = nn.LazyLinear(self.embedding_size)
        self.fc2 = nn.LazyLinear(1)
        self.fc3 = nn.LazyLinear(170)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p=self.dropout_rate)
        self.position_encoding = PositionalEncoding1D(self.embedding_size)
    
    def make_src_mask(self, src):
        src_mask = src == 0 # padding idx
        # return src_mask
        return src_mask.to(self.device) # (batch_size, max_src_len)

  
    def forward(self, x0, x1, masks, v_masks):
        seq, intervals = torch.tensor(x0), torch.tensor(x1)
        # print("seq shape", seq.shape)
        # print("interval shape", intervals.shape)

        batch_size = seq.shape[0]
        src_emb = self.embedding(seq)
        e = self.flatten_2(src_emb)
        # print("e.shape", e.shape)
        # input mask, reshape 3 dimension to 2
        e_mask = self.flatten_1(masks)
        # print("e_mask.shape", e_mask.shape) 

        # Vanilla Encoder
        h = self.encoder_stack(e, e_mask)
        # Attention pooling
        # print("before pooling", h.shape)
        v = self.attention_pooling(h)
        # print("after pooling", v.shape)
        
        e_p = self.interval_embedding(intervals)
        e_p = self.flatten_1(e_p)
        

        v = v + e_p
        # print("v", v.shape)
        v = v.view(batch_size, -1, self.embedding_size)
        
        h2 = self.encoder_stack(v, v_masks)
        # print("before pooling 2", h2.shape)
        out = self.attention_pooling(h2)
        # print("after pooling", out.shape)

        out = self.fc1(out)
        out = self.relu(out)
        # print("after relu", out.shape)

        
        out = self.dropout(out)
        out = self.fc3(out)
        
        probs = self.sigmoid(out)
        # print("after sigmoid", probs.shape)
        return probs
        return probs.view((batch_size, 170))
bite_net = BiteNet(dataset=dataset)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [23]:
import torch.optim as optim

criterion = nn.BCELoss()
optimizer = optim.Adam(bite_net.parameters(), lr=0.001)


In [27]:
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, precision_recall_curve, auc
def eval_model(model, val_loader):
    
    model.eval()
    y_pred = torch.LongTensor()
    y_score = torch.Tensor()
    y_true = torch.LongTensor()
    model.eval()
    for x0, x1, masks, v_masks, y0, y1 in val_loader:
        y_hat = model(x0, x1, masks, v_masks)
        # print(y)
        # print(y_hat)
        
        # print("y_hat", y_hat)
        y_score = torch.cat((y_score,  y_hat.detach().to(device)), dim=0)
        y_hat = (y_hat > 0.5).int()
        y_pred = torch.cat((y_pred, y_hat.detach().to(device)), dim=0)
        y_true = torch.cat((y_true, y1.detach().to(device)), dim=0)
        
    torch.set_printoptions(profile="full")
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, average="micro")
    # roc_auc = roc_auc_score(y_true, y_score, average='micro')

    # precision, recall, thresholds = precision_recall_curve(y_true, y_score)
    # Use AUC function to calculate the area under the curve of precision recall curve
    # auc_precision_recall = auc(recall, precision)
    # auc_precision_recall = 0
    return p, r, f

In [28]:
def train(model, train_loader, val_loader, n_epochs):
    for epoch in range(n_epochs):
      model.train()
      train_loss = 0
      for x0, x1, masks, v_masks, y0, y1 in train_loader:
        optimizer.zero_grad()
        y_pred = model(x0, x1, masks, v_masks)

        loss = criterion(y_pred.squeeze(), y1)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
      train_loss = train_loss / len(train_loader)
      print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
      p, r, f = eval_model(model, val_loader)
      print('Epoch: {} \t Validation p: {:.4f}, r:{:.4f}, f: {:.4f}'
              .format(epoch+1, p, r, f))
      

In [ ]:
n_epochs = 10
train(bite_net, train_loader, val_loader, n_epochs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:

""